In [2]:
import pandas as pd
import numpy as np
import math
import os
from PyRadioLoc.Utils.GeoUtils import GeoUtils
%run utils.ipynb

In [3]:
FILES = [
    './result/teorico_med.csv',
    './result/teorico_fingerprint_20.csv',
    './result/teorico_fingerprint_30.csv',
    './result/teorico_med_fingerprint_20.csv',
    './result/teorico_med_fingerprint_30.csv'
]

# 1. Análise dos dados

In [5]:
data = pd.read_csv("./database/LocTreino_Equipe_4.csv")
round(data.describe(), 4)

,lat,lon,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,pontoId
count,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000
mean,-8.0522,-34.9523,-88.8943,-92.6742,-89.1011,-88.5829,-90.5873,-89.2747,-87.8370,-91.0903,-93.1732,4.6173,3.9620,3.4467,8134.6800
std,0.0032,0.0040,6.2446,5.7099,6.1293,12.7005,7.9780,6.9039,7.8992,9.9822,8.0298,1.4671,1.6483,1.7569,889.6532
min,-8.0596,-34.9599,-110.2100,-105.0000,-104.4850,-107.0100,-104.4700,-104.5400,-113.3000,-115.6300,-105.1300,1.0000,1.0000,0.0000,6616.0000
25%,-8.0551,-34.9552,-92.8762,-97.1029,-93.0537,-97.8833,-96.7362,-94.3000,-92.2300,-98.0980,-97.2550,3.0000,3.0000,2.0000,7360.7500
50%,-8.0520,-34.9527,-89.4886,-93.6064,-90.1814,-93.1467,-91.9350,-90.6569,-89.3700,-94.3320,-96.2812,5.0000,4.0000,3.0000,8158.5000
75%,-8.0495,-34.9485,-85.5675,-89.4000,-86.2739,-83.1375,-86.8605,-85.6389,-84.1700,-85.3800,-93.3875,6.0000,5.0000,5.0000,8924.2500
max,-8.0464,-34.9451,-61.3800,-65.3900,-62.3200,-36.9600,-55.5500,-60.4800,-52.6900,-55.8700,-50.1000,8.0000,7.0000,7.0000,9679.0000


# 2. Histograma dos erros (em metros)

# 3. Boxplot dos erros (em metros)

# 4. Mapa de comparação - posições preditas vs. posições reais

# 5. Erros de localização médio, mínimo, máximo e desvio padrão

## 5.1 Erro modelo teórico e com medições

In [6]:
def calculate_errors(df_real, df_predict):
    """
    Retorna uma lista contendo os erros em metros de todos os pontos no real e na predição
    """
    df_merged = merge_on_pontoId(df_predict, df_real)
    errors = []
    
    for idx, point in df_merged.iterrows():
        distance = GeoUtils.distanceInKm(point.lat_x, point.lon_x, point.lat_y, point.lon_y)
        errors.append(round(distance * 1000, 2))
        
    return errors

def calculate_theorical_errors():
    df_real = pd.read_csv("./database/LocTreino_Equipe_4.csv")
    dict_errors = {}
    
    for file in FILES:
        filename = file.split("/")
        filename = "/".join(filename[2:]).replace(".csv", "")
        df_predict = pd.read_csv(file)
        dict_errors[filename] = calculate_errors(df_real, df_predict)
    
    return pd.DataFrame(dict_errors)

In [7]:
# Gera o dataframe contendo os erros
errors_df = calculate_theorical_errors()
round(errors_df.describe(), 2)

,teorico_med,teorico_fingerprint_20,teorico_fingerprint_30,teorico_med_fingerprint_20,teorico_med_fingerprint_30
count,150.00,150.00,150.00,150.00,150.00
mean,58.26,837.39,833.12,58.26,58.26
std,86.41,308.27,309.19,86.41,86.41
min,1.56,89.41,78.55,1.56,1.56
25%,7.88,638.14,639.76,7.88,7.88
50%,24.38,790.89,789.35,24.38,24.38
75%,54.05,1022.47,1021.84,54.05,54.05
max,458.76,1535.82,1527.27,458.76,458.76
